In [31]:
import os
#os.system('pip install numpy pandas datetime scipy sklearn colorama sklearn')
import numpy as np
import pandas as pd
import math
import datetime
import scipy
from scipy import stats

finaly = pd.DataFrame(columns = [str(i) for i in range(1, 38)])

path = "/home/mrred/Загрузки/Demonstration_D_S.xlsx"
data = pd.read_excel(path)
dat = data.copy()
q25 = dat[['Height', 'Weight']].quantile(0.25)
q75 = dat[['Height', 'Weight']].quantile(0.75)
IQR = q75 - q25
cut_off = IQR * 1.5
cut_off
lower, upper = q25 - cut_off, q75 + cut_off

dat_st = data[['Height']].copy()
dat_st[dat_st['Height'] > upper['Height']] = None
dat_st[dat_st['Height'] < lower['Height']] = None

dat[dat['Height'] > upper['Height']] = None
dat[dat['Height'] < lower['Height']] = None
dat[dat['Weight'] > upper['Weight']] = None
dat[dat['Weight'] < lower['Weight']] = None
dat['BMI'] = dat['Weight'] / (dat['Height']*0.01)**2
dat = dat[dat['Subsample']==1]
#q25_bmi = dat[['BMI']].quantile(0.25)
#q75_bmi = dat[['BMI']].quantile(0.75)
#IQR_bmi = q75_bmi - q25_bmi
#cut_off_bmi = IQR_bmi * 1.5
#cut_off_bmi
#lower_bmi, upper_bmi = q25_bmi - cut_off_bmi, q75_bmi + cut_off_bmi
#dat[dat['BMI'] > upper_bmi['BMI']] = None
#dat[dat['BMI'] < lower_bmi['BMI']] = None

In [32]:
last = dat[dat['Subsample']==1].copy()
last = last[["BMI", "LDDD", "Physical activity"]]
last = last.rename(columns={"Physical activity": "PA"})
last['LDDD'] = last['LDDD'].replace(1.0, 2.0)
last['LDDD'] = last['LDDD'].replace(0.0, 1.0)
last

,BMI,LDDD,PA
8,24.486647,2.0,Low
10,24.626620,1.0,Moderate
11,24.604133,1.0,Low
15,24.672679,2.0,Moderate
16,24.653244,1.0,Low
...,...,...,...
259,29.673674,2.0,Low
260,29.812254,1.0,Moderate
261,29.933027,2.0,High
262,30.669969,1.0,Moderate


In [48]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('BMI ~ C(LDDD) + C(PA) + C(LDDD) : C(PA)', data=last).fit()
result = sm.stats.anova_lm(model, typ=2)
  
# Print the result
result

,sum_sq,df,F,PR(>F)
C(LDDD),0.971168,1.0,0.811910,0.369361
C(PA),0.442134,2.0,0.184815,0.831494
C(LDDD):C(PA),3.444000,2.0,1.439617,0.241082
Residual,143.538130,120.0,NaN,NaN


In [10]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

In [11]:
eta_squared(result)
omega_squared(result)
result.round(3)

,df,sum_sq,mean_sq,F,PR(>F),eta_sq,omega_sq
C(LDDD),1.0,0.934,0.934,0.781,0.379,0.006,-0.002
C(PA),2.0,0.442,0.221,0.185,0.831,0.003,-0.013
C(LDDD):C(PA),2.0,3.444,1.722,1.440,0.241,0.023,0.007
Residual,120.0,143.538,1.196,NaN,NaN,NaN,NaN


In [12]:
#pip install pingouin

last.dropna()

,BMI,LDDD,PA
8,24.486647,1.0,Low
10,24.626620,0.0,Moderate
11,24.604133,0.0,Low
15,24.672679,1.0,Moderate
16,24.653244,0.0,Low
...,...,...,...
259,29.673674,1.0,Low
260,29.812254,0.0,Moderate
261,29.933027,1.0,High
262,30.669969,0.0,Moderate


In [20]:
import pingouin as pg
anv = pg.anova(dv='BMI', between=['LDDD', 'PA'], data=last, detailed=False).round(3)

In [21]:
anv

,Source,SS,DF,MS,F,p-unc,np2
0,LDDD,0.971,1.0,0.971,0.812,0.369,0.007
1,PA,0.442,2.0,0.221,0.185,0.831,0.003
2,LDDD * PA,3.444,2.0,1.722,1.440,0.241,0.023
3,Residual,143.538,120.0,1.196,NaN,NaN,NaN


In [46]:
from pingouin import mixed_anova, read_dataset
df = read_dataset('mixed_anova')
aov = pg.mixed_anova(dv='Scores', within='Time', between='Group', subject='Subject', data=df)
aov

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,Group,5.459963,1,58,5.459963,5.051709,0.028420,0.080120,NaN
1,Time,7.628428,2,116,3.814214,4.027394,0.020369,0.064929,0.998751
2,Interaction,5.167192,2,116,2.583596,2.727996,0.069545,0.044922,NaN


In [47]:
df

,Scores,Time,Group,Subject
0,5.971435,August,Control,0
1,4.309024,August,Control,1
2,6.932707,August,Control,2
3,5.187348,August,Control,3
4,4.779411,August,Control,4
...,...,...,...,...
175,6.176981,June,Meditation,55
176,8.523692,June,Meditation,56
177,6.522273,June,Meditation,57
178,4.990568,June,Meditation,58


In [53]:
round(anv[anv["Source"] == "PA"]["p-unc"].values[0], 4)

0.0075

In [8]:
last_repl = last.copy()
last_repl[["PA"]] = last_repl[["PA"]].replace({'Low': 1, 'Moderate': 2, 'High': 3})
last_repl

,BMI,LDDD,PA
21,26.033168,1.0,3
23,26.104898,0.0,3
24,25.931796,1.0,3
27,26.031497,1.0,2
31,25.971918,0.0,2
...,...,...,...
240,28.485904,1.0,1
243,28.376112,1.0,3
244,28.458744,0.0,1
249,28.765307,1.0,1


In [9]:
anv_repl = pg.anova(dv='BMI', between=['LDDD', 'PA'], data=last_repl.dropna(), detailed=True)

In [10]:
anv_repl

,Source,SS,DF,MS,F,p-unc,np2
0,LDDD,0.117000,1.0,0.117000,0.233166,0.630173,0.002174
1,PA,1.921074,2.0,0.960537,1.914223,0.152472,0.034544
2,LDDD * PA,0.084492,2.0,0.042246,0.084191,0.919316,0.001571
3,Residual,53.691484,107.0,0.501790,NaN,NaN,NaN


In [76]:
dnn

,IID,FID,Sex,Date of birth,Weight,Height,Body fat mass,Physical activity,Ethnicity,Date of visit,Maximum degree of disk degeneration in lumbar spine,LDDD,Chronic low back pain,Subsample,BMI
0,1,1,Female,1954-06-27,19.37,112.7,38,Low,Russian,2021-08-31,4,1,1,1,15.250419
1,2,2,Male,1949-01-13,20.43,114.9,30,Low,Russian,2021-10-27,4,1,1,1,15.474916
2,3,3,Male,1939-02-21,47.95,116.3,30,High,Russian,2020-01-18,4,1,1,0,35.451056
3,4,4,Male,1986-07-06,50.30,150.0,23,Moderate,Russian,2019-12-13,3,0,0,0,22.355556
4,5,5,Female,1969-08-04,51.69,150.1,36,Moderate,Russian,2019-05-08,4,1,1,0,22.942733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,273,273,Female,2002-02-14,128.04,184.1,33,High,Chechen,2019-05-16,3,0,0,1,37.777924
273,274,274,Female,1936-06-06,131.44,185.4,38,Moderate,Russian,2021-05-17,5,1,1,1,38.239138
274,275,275,Female,1980-06-06,NaN,213.5,34,Moderate,Russian,2020-09-21,5,1,1,1,NaN
275,276,276,Male,1943-11-22,NaN,216.9,30,Low,Russian,2021-03-08,3,0,1,0,NaN


In [2]:
import pandas as pd
data = pd.read_excel("/home/mrred/Загрузки/Demonstration_D_S.xlsx")
data

,IID,FID,Sex,Date of birth,Weight,Height,Body fat mass,Physical activity,Ethnicity,Date of visit,Maximum degree of disk degeneration in lumbar spine,LDDD,Chronic low back pain,Subsample
0,1,1,Female,1954-06-27,19.37,112.7,38,Low,Russian,2021-08-31,4,1,1,1
1,2,2,Male,1949-01-13,20.43,114.9,30,Low,Russian,2021-10-27,4,1,1,1
2,3,3,Male,1939-02-21,47.95,116.3,30,High,Russian,2020-01-18,4,1,1,0
3,4,4,Male,1986-07-06,50.30,150.0,23,Moderate,Russian,2019-12-13,3,0,0,0
4,5,5,Female,1969-08-04,51.69,150.1,36,Moderate,Russian,2019-05-08,4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,273,273,Female,2002-02-14,128.04,184.1,33,High,Chechen,2019-05-16,3,0,0,1
273,274,274,Female,1936-06-06,131.44,185.4,38,Moderate,Russian,2021-05-17,5,1,1,1
274,275,275,Female,1980-06-06,NaN,213.5,34,Moderate,Russian,2020-09-21,5,1,1,1
275,276,276,Male,1943-11-22,NaN,216.9,30,Low,Russian,2021-03-08,3,0,1,0


In [22]:
import scipy
real_2010 = {
    'Armenian': 0.84,#
    'Bashkir': 1.26,#
    'Tatar': 4.21,#
    'Ukrainian': 1.53,#
    'German': 0.32,#
    'Chuvash': 1.15,#
    'Chechen': 1.13,#
    'Kazakh': 0.51,#
    'Yakut': 0.38,#
    'Russian': 87.97,#
    'Buryat': 0.36,#
    'Uzbek': 0.24,#
}
dft = pd.DataFrame(real_2010.items())
dft = dft.rename(columns = {0: '', 1: 'Ethnicity_2010'})
dft = dft.set_index('')
a = pd.concat([dft, data['Ethnicity'].value_counts()], axis=1)
a

,Ethnicity_2010,Ethnicity
Armenian,0.84,NaN
Bashkir,1.26,4.0
Tatar,4.21,11.0
Ukrainian,1.53,6.0
German,0.32,1.0
Chuvash,1.15,1.0
Chechen,1.13,3.0
Kazakh,0.51,3.0
Yakut,0.38,NaN
Russian,87.97,246.0


In [23]:
a['Ethnicity'] = ((100. * a.Ethnicity) / a.Ethnicity.sum())
a['Ethnicity'] = round(a['Ethnicity']/100, 4)
a['Ethnicity_2010'] = a['Ethnicity_2010']/100


,Ethnicity_2010,Ethnicity
Armenian,0.0084,NaN
Bashkir,0.0126,0.0144
Tatar,0.0421,0.0397
Ukrainian,0.0153,0.0217
German,0.0032,0.0036
Chuvash,0.0115,0.0036
Chechen,0.0113,0.0108
Kazakh,0.0051,0.0108
Yakut,0.0038,NaN
Russian,0.8797,0.8881


In [38]:
b = pd.DataFrame(a.Ethnicity_2010 * data['Ethnicity'].value_counts().sum())
c = pd.DataFrame(data['Ethnicity'].value_counts())
red = pd.concat([b, c], axis=1).fillna(0)
nexp_2010 = ((red.Ethnicity-red.Ethnicity_2010)**2)/red.Ethnicity_2010
nexp_2010

Armenian     2.326800
Bashkir      0.074465
Tatar        0.037546
Ukrainian    0.732472
German       0.014559
Chuvash      1.499422
Chechen      0.005407
Kazakh       1.783479
Yakut        1.052600
Russian      0.022147
Buryat       0.000008
Uzbek        0.169012
dtype: float64

In [39]:
p_2010 = scipy.stats.chi2.sf(nexp_2010, 11)
p_2010

array([0.99697864, 1.        , 1.        , 0.99998983, 1.        ,
       0.99962033, 1.        , 0.99912393, 0.99993465, 1.        ,
       1.        , 1.        ])

In [54]:
chi2, p, dof, expected = scipy.stats.chi2_contingency(red, correction=True)

In [56]:
chi2

5.536760731439471

In [59]:
def categories(series):
    return range(int(series.min()), int(series.max()) + 1)

def chi_square_of_df_cols(df, col1, col2):
    df_col1, df_col2 = df[col1], df[col2]
    cats1, cats2 = categories(df_col1), categories(df_col2)

    def aux(is_cat1):
        return [sum(is_cat1 & (df_col2 == cat2))
                for cat2 in cats2]

    result = [aux(df_col1 == cat1)
              for cat1 in cats1]

    return scipy.stats.chi2_contingency(result)

aa = a.fillna(0)
chi_square_of_df_cols(aa, 'Ethnicity', 'Ethnicity_2010')

/home/mrred/.local/lib/python3.10/site-packages/scipy/stats/contingency.py:128: RuntimeWarning: invalid value encountered in divide
  expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)


(0.0, 1.0, 0, array([[nan]]))

In [60]:
aa

,Ethnicity_2010,Ethnicity
Armenian,0.0084,0.0000
Bashkir,0.0126,0.0144
Tatar,0.0421,0.0397
Ukrainian,0.0153,0.0217
German,0.0032,0.0036
Chuvash,0.0115,0.0036
Chechen,0.0113,0.0108
Kazakh,0.0051,0.0108
Yakut,0.0038,0.0000
Russian,0.8797,0.8881
